# ERS NIRCam Demo with Eureka!


## Authors: Megan Mansfield and Eva-Maria Ahrer
 Hello! This notebook shows a full demonstration reducing and analyzing the NIRCam/F322W2 transit of WASP-39b from the JWST Transiting Exoplanet Early Release Science Program (ERS 1366). Following this pipeline, you should be able to reproduce the NIRCam spectrum presented in the main text of Ahrer et al. (2022). 
 
This data reduction uses the [Eureka!](https://github.com/kevin218/Eureka) pipeline. Before running this Jupyter Notebook, please follow the steps in "README.md" to set up an environment for this demonstration and install Eureka. Note that this demonstration is using a fixed version of Eureka! to ensure that no future updates break the example here, so even if you have the current version of Eureka! installed on your own machine you'll need to follow the set-up steps to ensure this demo works properly. For more information on Eureka! see its documentation or refer to Bell et al. (2022).

### First, download the data set.
#### Data can be downloaded from MAST at: (DOI? Specify which file extensions/stage?)
After downloading the data, place it in a directory "./Uncalibrated" within this tutorial.

### Next, let's import packages!

In [1]:
import sys
sys.path.append('./')
sys.path.insert(0,'./')
import eureka.S1_detector_processing.s1_process as s1
import eureka.S2_calibrations.s2_calibrate as s2
import eureka.S3_data_reduction.s3_reduce as s3
import eureka.S4_generate_lightcurves.s4_genLC as s4
import eureka.S5_lightcurve_fitting.s5_fit as s5
import eureka.S6_planet_spectra.s6_spectra as s6

from eureka.lib import readECF

eventlabel = 'nircam_wasp39b'
ecf_path = './ecf/'

2022-10-03 12:27:55,986 - stpipe - WARNING - No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


#### Note: Eureka! is divided into six "Stages". Each Stage has a corresponding Eureka! Control File or .ecf file, and in Stage 5 there's also a Eureka! Parameter File or .epf. 
These files are named "S2_eventlabel.ecf" where the first half of the name (S2, S3, etc.) refers to the Stage that .ecf file interacts with, and the second half refers to the "eventlabel" keyword we defined above. See the Eureka! documentation for a full description of each stage and all keywords in the .ecf files. Here we'll give a very brief summary of each Stage and discuss a few important keywords for reproducing the paper results.

# Stage 1: Correcting detector-level effects and fitting the up-the-ramp slope.
#### Most important keywords in the .ecf:
1. jump_rejection_threshold - this sets the sigma threshold for rejecting a jump in the up-the-ramp slope as due to a cosmic ray hit. The standard value in the jwst pipeline is 4.0, but this data reduction found a value of 6.0 produced better results.
2. topdir - Edit this to make it the path to where you've downloaded the data for this demo. The path should look like this: /path/to/NIRCam_demo/wasp39b_data/
3. inputdir - This keyword tells Eureka! where to look for the Uncalibrated outputs to feed into the Stage 1 code. This path is relative to "topdir". In this demo, we set inputdir=/Uncalibrated
4. outputdir - This is where Eureka! will save all the Stage 2 outputs, including plots and log files. Here we set outputdir=/Stage1, which means the output files will be saved within a folder on the path /path/to/NIRCam_demo/NIRCam_full_data/Stage1/
#### Note that topdir, inputdir, and outputdir work the same in every .ecf file, so in each file edit them so that "inputdir" points to the previous Stage's outputs, and "outputdir" points to where you want to save that Stage's outputs.

In [4]:
ecffile = 'S1_' + eventlabel + '.ecf'
meta = readECF.MetaClass(ecf_path, ecffile)

meta.jump_rejection_threshold = 4.0
meta.topdir = '/path/to/NIRCam_demo/wasp39b_data/'
meta.inputdir = '/Uncalibrated'
meta.outputdir = '/Stage1'

meta.write(ecf_path)


## for Megan: add these lines to meta.write(ecf_path) to update meta.lines (self.lines) before it writes the .ecf file
# for i in range(len(meta.lines)):
#     line = meta.lines[i]
#     # Strip off comments:
#     if "#" in line:
#         line = line[0:line.index('#')]
#     line = line.strip()

#     # Keep only useful lines:
#     if len(line) > 0:
#         name = line.split()[0]
#         val = ''.join(line.split()[1:])
#         new_val = meta.params[name]
#         if val != new_val:
#             #print(str(val),str(new_val))
#             #print(meta.lines[i])
#             #print(i)
#             meta.lines[i] = meta.lines[i].replace(str(val),str(new_val))
#             #print(meta.lines[i])

### And that's it! Let's run Stage 1!

In [6]:
s1_meta = s1.rampfitJWST(eventlabel, ecf_path=ecf_path) 

PermissionError: You do not have the permissions to make the folder /path/to/NIRCam_demo/wasp39b_data/Stage1/S1_2022-10-03_nircam_wasp39b_run1/
Your topdir is currently set to/path/to/NIRCam_demo/wasp39b_data/, but your user account is called phrgmk.
You likely need to update the topdir setting in your S1 .ecf file.

# Stage 2: Additional pre-spectral-extraction steps, such as assignment of the world coordinate system, flat fielding, wavelength calibration. Outputs calibrated 2D images.
#### Most important keywords in the .ecf:
1. skip_bkg_subtract - Kevin had this set to false?? Check this.


In [8]:
ecffile = 'S2_' + eventlabel + '.ecf'
meta = readECF.MetaClass(ecf_path, ecffile)
meta.skip_bkg_subtract = True
meta.hide_plots = True
meta.write(ecf_path)

### Let's run Stage 2!

In [ ]:
s2_meta = s2.calibrateJWST(eventlabel, ecf_path=ecf_path, s1_meta=s1_meta)

# Stage 3: Identify source position, perform background subtraction, perform spectral extraction to produce time series of 1D extracted spectra.
#### Most important keywords in the .ecf:
1. ncpu - Number of CPUs on your machine. This won't affect the data reduction except to make it run faster if you are able to run it on more CPUs. You can change this number to fit whatever machine you're running on.
2. ywindow and xwindow - These specify the region of the image that you're interested in performing background subtraction and spectral extraction on. Note that this is the *full* image, not just the window surrounding the spectral trace. For this demo, the ywindow is set to ignore reference pixels on the edges of the detector, and the xwindow is set to select the region of higher throughput where the spectral trace can actually be seen in the image. These values were selected based on viewing an image in ds9 or another fits file viewer and looking at the position of the spectral trace.

#### The next several parameters have to do with the background subtraction. Eureka! performs background subtraction by identifying the spectral trace, masking out a region surrounding the spectral trace, and using the remaining pixels as the background.

3. bg_hw - Defines the half-width of the masked area not included in background subtraction. In this example, a value of 14 means that 14 pixels both above and below the identified source position are excluded from each column. We tested a few different values and found that 14 minimized the MAD of the resulting light curves, as it was large enough to exclude contamination from the spectrum but not so large that the background subtraction was affected by having less pixels to estimate the background from.
4. bg_deg - The background is subtracted by doing a column-by-column polynomial fit, and bg_deg defines the degree of that fit. Setting bg_deg = -1 will just calculate and subtract out a median for each column. For this reduction, bg_deg=1 removed the background sufficiently well.

#### Now some parameters for how we'll do the spectral extraction!
5. spec_hw - Defines the half-width of the region you extract the spectrum from. In this example, a value of 9 means that the extraction will be perfomed on a box of pixels extending 9 up and 9 down from the identified source position. We tested several values and found that a half-width of 9 minimized the MAD of the resulting light curves, as it was wide enough to catch the edges of the spectrum but not so wide that it added too much background contamination.

#### Finally, some parameters for printing diagnostics and saving output.
6. isplots_S3 - How many plots do you want to create? This can be set to 1, 3, or 5, where a bigger number will print out more different types of diagnostic plots. The default in this demo is 5 so that you can see all the diagnostics.

In [10]:
ecffile = 'S3_' + eventlabel + '.ecf'
meta = readECF.MetaClass(ecf_path, ecffile)

meta.ncpu = 1
meta.xwindow = [4,64]
meta.ywindow = [4,1704]

meta.bg_hw = 14
meta.bg_deg = 1

meta.spec_hw = 9

meta.isplots_S3 = 3
meta.hide_plots = True

meta.write(ecf_path)

In [ ]:
s3_spec, s3_meta = s3.reduce(eventlabel, ecf_path=ecf_path, s2_meta=s2_meta)

# Stage 4: Convert 1D extracted spectra to time series light curves

#### Most important keywords in the .ecf:
1. nspecchan - number of spectroscopic channels
2. compute_white - whether to compute the white-light lightcurve
3. wave_min and wave_max - the wavelength range in micron
4. clip_unbinned and clip_binned - whether to perform sigma-clipping on the unbinned and/or binned 1D time series
5. sigma - the number of sigmas a point must be from the rolling median to be considered an outlier

#### Limb-darkening coefficients using exotic-ld (https://exotic-ld.readthedocs.io/en/latest/)
6. compute_ld - whether to compute the limb-darkening coefficients with exotic-ld
7. exotic_ld_direc - directory for ancillary files for exotic-ld (see documentation of exotic-ld)
8. exotic_ld_grid - whether to use 3D or 1D stellar model grids
9. exotic_ld_file - if you want to use custom throughput file (leave empty if using default)

In [12]:
ecffile = 'S4_' + eventlabel + '.ecf'
meta = readECF.MetaClass(ecf_path, ecffile)


meta.nspecchan = 110
meta.wave_min = 2.405
meta.wave_max = 4.055

meta.compute_white = True 

meta.clip_unbinned = False   
meta.clip_binned = True    
meta.sigma = 4     

# Limb-darkening parameters needed to compute exotic-ld
meta.compute_ld = False
meta.exotic_ld_direc = '/data/exotic-ld_data/' 
meta.exotic_ld_grid = '3D' 
meta.exotic_ld_file = '/data/exotic-ld_data/NIRCam_throughput_full.csv' 


meta.write(ecf_path)

In [ ]:
s4_spec, s4_lc, s4_meta = s4.genlc(eventlabel, ecf_path=ecf_path, s3_meta=s3_meta)

# Stage 5: Light Curve Fitting

#### Most important keywords in the .ecf:
1. fit_method - Which fitting method to use, options are: lsq, emcee, dynesty (can list multiple types separated by commas)
2. run_myfuncs - What does the fit consist of? Options are: batman_tr (transit model), batman_ecl (eclipse model), sinusoid_pc (phase curve), expramp (exponential ramp), polynomial, step, and GP (Gaussian Process). Must list all models you want to use!
3. use_generate_ld - Whether to use the 'exotic-ld' limb-darkening coefficients generated in Stage 4

In [14]:
ecffile = 'S5_' + eventlabel + '.ecf'
meta = readECF.MetaClass(ecf_path, ecffile)

meta.fit_method = '[emcee]'               #options are: lsq, emcee, dynesty (can list multiple types separated by commas)
meta.run_myfuncs = '[batman_tr,polynomial]'  #options are: batman_tr, batman_ecl, sinusoid_pc, expramp, polynomial, step, and GP (can list 

meta.use_generate_ld = 'exotic-ld' #use the generated limb-darkening coefficients from Stage 4?

meta.write(ecf_path)

#### Specific fitter parameters are in the cell below
Least-squares (lsq)
1. lsq_method - the scipy.optimize.minimize optimization method to use

Markov Chain Monte Carlo (MCMC)
1. lsq_first - whether to run a least-squares fit first and use as initial values for MCMC fit
2. run_steps - number of steps
3. run_nwalkers - number of walkers
4. run_nburn - number of run_nsteps should be discarded as burn-in steps

Dynesty (nested sampling)
1. run_nlive - number of live points
2. run_tol - tolerance value i.e. convergence criterion

In [ ]:
#fitter parameters
#lsq
meta.lsq_method = 'Powell'

#mcmc
meta.lsq_first = True    
meta.run_nsteps = 500
meta.run_nwalkers = 100
meta.run_nburn = 100     

#dynesty
meta.run_nlive = 1024    
meta.run_tol = 0.1

meta.write(ecf_path)


## Important: Check your Parameter file (.epf file) for all parameter specific inputs e.g. prior types and ranges



### Let's fit some light curves!

In [ ]:
s5_meta = s5.fitlc(eventlabel, ecf_path=ecf_path, s4_meta=s4_meta)

# Stage 6: Create Final Spectrum

#### Most important keywords in the .ecf:
1. y_unit - For plotting the transmission spectrum, options include Rp/Rs, (Rp/Rs)^2, Fp/Fs
2. y_sclar - Can be used to convert to ppm (1e6), percent (100), etc.

In [ ]:
ecffile = 'S6_' + eventlabel + '.ecf'
meta = readECF.MetaClass(ecf_path, ecffile)

meta.y_unit = '(Rp/Rs)^2' 
meta.y_scalar = 100 

meta.write(ecf_path)

### Let's run the final stage to get a transmission spectrum!

In [ ]:
s6_meta = s6.plot_spectra(eventlabel, ecf_path=ecf_path, s5_meta=s5_meta)